In [50]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

In [51]:
tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-j-6B")
model = AutoModelForCausalLM.from_pretrained("EleutherAI/gpt-j-6B", torch_dtype=torch.float16)

Some weights of the model checkpoint at EleutherAI/gpt-j-6B were not used when initializing GPTJForCausalLM: ['transformer.h.0.attn.bias', 'transformer.h.0.attn.masked_bias', 'transformer.h.1.attn.bias', 'transformer.h.1.attn.masked_bias', 'transformer.h.10.attn.bias', 'transformer.h.10.attn.masked_bias', 'transformer.h.11.attn.bias', 'transformer.h.11.attn.masked_bias', 'transformer.h.12.attn.bias', 'transformer.h.12.attn.masked_bias', 'transformer.h.13.attn.bias', 'transformer.h.13.attn.masked_bias', 'transformer.h.14.attn.bias', 'transformer.h.14.attn.masked_bias', 'transformer.h.15.attn.bias', 'transformer.h.15.attn.masked_bias', 'transformer.h.16.attn.bias', 'transformer.h.16.attn.masked_bias', 'transformer.h.17.attn.bias', 'transformer.h.17.attn.masked_bias', 'transformer.h.18.attn.bias', 'transformer.h.18.attn.masked_bias', 'transformer.h.19.attn.bias', 'transformer.h.19.attn.masked_bias', 'transformer.h.2.attn.bias', 'transformer.h.2.attn.masked_bias', 'transformer.h.20.attn.bi

In [59]:
import time

def execution_time(func):
    def wrapper(*args, **kwargs):
        start_time = time.time() 
        result = func(*args, **kwargs)
        end_time = time.time()
        execution_duration = end_time - start_time
        print(f"Execution time for {func.__name__}: {execution_duration:.4f} seconds")
        return result
    return wrapper

In [60]:
@execution_time
def generate_answer(query, information):
    try:
        system_prompt = (
            "You are a knowledgeable assistant specializing in movies and well-known individuals. Your task is to answer "
            "the following question as accurately and concise as possible, based on the provided context."
        )
                
        context_parts = [system_prompt, f"Question: {query}\n\n", "Retrieved Information:\n\n"]
        
        for entity_id, data in information.items():
            entity_name = data.get('entity_name', 'Unknown')
            context_parts.append(f"Entity: {entity_name} ({entity_id})\n")
            for prop, values in data.items():
                values = values if isinstance(values, list) else [values]
                val_str = ', '.join(values)
                context_parts.append(f" - {prop}: {val_str}\n")
            context_parts.append("\n")
        
        context_parts.append("Answer:")
        context_str = ''.join(context_parts)
        
        inputs = tokenizer(context_str, return_tensors="pt").to(model.device)
        
        with torch.no_grad():
            outputs = model.generate(
                inputs["input_ids"],
                max_new_tokens = 70,
                num_return_sequences=1,       # Single answer
                early_stopping=True,          # Stop early if the model has generated a reasonable output
                temperature=1.0,              # Lower temperature for more focused answers
                top_p=0.9                     # Nucleus sampling for a more diverse yet concise response
            )
        
        answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
        answer = answer.split("Answer:")[-1].strip()
        return answer
    except Exception as e:
        print(f"Error generating answer: {e}")
        return "I'm sorry, I encountered an error while processing your request."

In [61]:
context = {
  "Q76364": {
    "entity_name": "Hans Zimmer",
    "native language": [
      "German"
    ],
    "occupation": [
      "musician"
    ],
    "described by source": [
      "Obalky knih.cz"
    ],
    "religion": [
      "Christianity"
    ],
    "nominated for": [
      "Academy Award for Best Original Musical or Comedy Score",
      "Academy Award for Best Original Dramatic Score",
      "Academy Award for Best Original Score"
    ],
    "languages spoken, written or signed": [
      "German"
    ],
    "award received": [
      "Stephen Hawking Medal For Science Communication",
      "Academy Award for Best Original Score"
    ],
    "place of birth": [
      "Frankfurt am Main"
    ],
    "country of citizenship": [
      "Germany"
    ],
    "instance of": [
      "human"
    ],
    "work location": [
      "Los Angeles"
    ]
  }
}

In [62]:
generate_answer("Where was Hans Zimmer born?", context)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Execution time for generate_answer: 22.7667 seconds


'Hans Zimmer was born in Frankfurt am Main, Germany.\n\nRetrieved Information:\n\nEntity: Hans Zimmer (Q76364)\n - native language: German\n - occupation: musician\n - described by source: Obalky knih.cz\n - religion: Christianity\n - nominated for: Academy Award for Best Original Musical'